# Kombination von Text und Metainformationen

Im letzten Teil hast du gelernt, wie man Metainformationen zu Texten auswertet. Nun wollen wir diese Informationen mit solchen aus den Texten selbst verbinden.

## Daten einlesen

Zunächst liest du wie gewohnt wieder die linguistisch analysierten Daten ein:

In [ ]:
!pip install textacy

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles \
                   WHERE datePublished<'2021-01-01' \
                   ORDER BY datePublished", sql, index_col="id", parse_dates=["datePublished"])

## Korrelation Anzahl Tokens mit Anzahl Sätzen

Eine gute Frage wäre nun, ob bei den Artikeln die Satzlänge mit der Anzahl der Tokens korreliert. Das kannst du mit einem sog. *Scatterplot* herausfinden:

In [ ]:
df.plot.scatter(x='no_tokens', y='no_sentences')

Etwas übersichtlicher wird das mit `seaborn`, weil das eine Korrelationslinie einzeichnen kann. Sehr nett sind auch die Histogramme in den jeweiligen Dimensionen:

In [ ]:
import seaborn as sns
sns.jointplot(x=df["no_tokens"], y=df["no_sentences"], kind="reg")

Mithilfe des sog. *Pearson R* kannst du die Korrelation berechnen:

In [ ]:
import scipy.stats
scipy.stats.pearsonr(df["no_tokens"], df["no_sentences"])

Ein Wert von 1 entspricht dabei eine *Korrelation*, -1 steht für eine *Antikorrelation*.

## Häufigkeit der Beiträge vs. Textlänge

Man könnte sich nun weiter fragen, ob Autoren längere Artikel schreiben, wenn sie weniger veröffentlichen. Hierfür muss du zunächst nach den Autoren aggregrieren und die Artikel zählen:

In [ ]:
count_length = df.groupby("author").agg({"url": "count", "no_tokens": "mean"}).rename(columns={"url": "count"})

Anschließend kannst du wieder einen *Scatterplot* zeichnen:

In [ ]:
sns.jointplot(x=count_length["no_tokens"], y=count_length["count"], kind="reg")

Das *Pearson R* zeigt dir eine sehr unspezifische *Antikorrelation*:

In [ ]:
scipy.stats.pearsonr(count_length["no_tokens"], count_length["count"])

## Artikellänge vs. Kommentare

Als nächstes fragst du dich, ob längere Artikel mehr zum Nachdenken anregen und damit auch mehr Kommentare erzeugen:

In [ ]:
sns.jointplot(x=df["no_tokens"].fillna(0).map(int), 
              y=df["commentCount"].fillna(0).map(int), kind="reg")

Das *Pearson R* zeigt nur eine schwache Korrelation:

In [ ]:
scipy.stats.pearsonr(df["no_tokens"].fillna(0).map(int), df["commentCount"].fillna(0).map(int))

## Artikellängen bei Autoren

Nun soll es um die Artikellängen bei den Top-Autoren gehen. Dazu bestimmst du zunächst die Top-Autoren:

In [ ]:
top_authors = df.groupby("author").agg({"url": "count"}).sort_values("url").tail(20).index.values

Anschließend selektierst du in dem `DataFrame` nur die Artikel, die von diesen Autoren geschrieben wurden:

In [ ]:
top_author_articles = df[df["author"].isin(top_authors)]
len(top_author_articles)

Das ist eine ganze Menge!

Eine gute Visualisierung dafür ist der sog. [Box-Plot](https://de.wikipedia.org/wiki/Box-Plot):

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 10))
sns.boxplot(y="author", x="no_tokens", data=top_author_articles, palette="viridis")

Die Darstellung lässt sich noch ein bisschen optimieren und nach dem Median sortieren:

In [ ]:
plt.figure(figsize=(8, 10))
labels = top_author_articles.groupby("author").agg({"no_tokens": "median"}).sort_values("no_tokens").index.values
sns.boxplot(y="author", x="no_tokens", data=top_author_articles, order=labels, palette="viridis")

## Autor-spezifische Wordclouds

In [ ]:
ben_schwan = df[df["author"] == "Ben Schwan"]

In [ ]:
mark_mantel = df[df["author"] == "Mark Mantel"]

In [ ]:
from collections import Counter
from wordcloud import WordCloud
from spacy.lang.de.stop_words import STOP_WORDS
import regex as re
def word_cloud_for_field(df, field):
    c = Counter([w for words in df[field] for w in re.split(r'\||\#', words)])
    for w in STOP_WORDS:
        c[w] = 0
    wc = WordCloud(background_color="white", max_words=100)
    wc.generate_from_frequencies(c)
    plt.figure(figsize=(12,12))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off");    

In [ ]:
word_cloud_for_field(ben_schwan, "nouns")

In [ ]:
word_cloud_for_field(mark_mantel, "nouns")

## Leistungsfähige Kombinationen

Die Kombination aus Metadaten und solchen, die aus Texten ermittelt werden können, ist sehr leistungsfähig. Deiner Kreativität sind hier wenig Grenzen gesetzt.

Alles, was du in dieser Phase schon auswerten kannst, macht dir die Arbeit mit den weiter fortgeschrittenen Techniken deutlich einfacher.